## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

K.tensorflow_backend._get_available_gpus()

input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
model=keras.applications.xception.Xception(include_top=False, weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None, classes=10)

'''Resnet 50 架構'''
#model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    #input_tensor=input_tensor,
                                    #pooling=None, classes=10)
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

## 添加層數

In [3]:
x = model.output

'''可以參考Cifar10實作章節'''
x = GlobalAveragePooling2D()(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(p=0.1)(x)

predictions = Dense(units=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


C:\Users\SeanKang\Anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
W1223 14:21:25.017072  4296 deprecation.py:506] From C:\Users\SeanKang\Anaconda3\envs\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model深度： 136


## 鎖定特定幾層不要更新權重

In [4]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


C:\Users\SeanKang\Anaconda3\envs\python37\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Training

In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)

W1223 14:21:42.719341  4296 module_wrapper.py:139] From C:\Users\SeanKang\Anaconda3\envs\python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1223 14:21:42.733354  4296 module_wrapper.py:139] From C:\Users\SeanKang\Anaconda3\envs\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W1223 14:21:42.795410  4296 deprecation.py:323] From C:\Users\SeanKang\Anaconda3\envs\python37\lib\site-packages\tensorflow_core\python\ops\math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1223 14:21:43.085675  4296 module_wrapper.py:139] From C:\Users\SeanKang\Anaconda3\envs\python37\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecat

Epoch 1/100
50000/50000 [==============================] - 77s 2ms/step - loss: 1.3985 - acc: 0.5129
Epoch 2/100
50000/50000 [==============================] - 64s 1ms/step - loss: 1.0281 - acc: 0.6470
Epoch 3/100
50000/50000 [==============================] - 67s 1ms/step - loss: 0.9079 - acc: 0.6872
Epoch 4/100
50000/50000 [==============================] - 71s 1ms/step - loss: 0.8276 - acc: 0.7150
Epoch 5/100
50000/50000 [==============================] - 70s 1ms/step - loss: 0.7433 - acc: 0.7448
Epoch 6/100
50000/50000 [==============================] - 66s 1ms/step - loss: 0.6814 - acc: 0.7669
Epoch 7/100
50000/50000 [==============================] - 66s 1ms/step - loss: 0.6128 - acc: 0.7887
Epoch 8/100
50000/50000 [==============================] - 66s 1ms/step - loss: 0.5576 - acc: 0.8086
Epoch 9/100
50000/50000 [==============================] - 61s 1ms/step - loss: 0.4912 - acc: 0.8308
Epoch 10/100
50000/50000 [==============================] - 67s 1ms/step - loss: 0.4532 - a